In [477]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression

In [547]:
book_df = pd.read_pickle("data/book_data.pkl")
# book_df[['pub_date', 'og_pub_date']] = book_df[['pub_date','og_pub_date']].fillna(0)
book_df.describe()

,book_num,followers,pub_date,og_pub_date,avg_rating,pages,perc_like,trivia,quote_likes,rev_likes,num_revs,num_ratings,kindle_price,amzn_price,total_added,total_to_read,avg_added,avg_to_read
count,7.178000e+03,7178.000000,319.000000,166.000000,7178.000000,307.000000,7178.000000,7178.000000,7178.000000,7178.000000,7178.000000,7.178000e+03,58.000000,13.000000,7.178000e+03,7.178000e+03,7178.000000,7178.000000
mean,3.912209e+06,85.043188,1993.626959,1979.933735,0.140670,255.371336,3.312483,0.000836,4.632070,0.664113,22.017275,8.183044e+02,14.789483,18.779231,1.215412e+03,2.770836e+02,4.062330,1.382467
std,2.498723e+06,3491.525065,41.927333,65.204493,0.734739,237.090725,17.279788,0.052782,180.105361,21.898843,1027.394086,3.825757e+04,18.381087,17.040804,5.740476e+04,1.397727e+04,65.378510,18.199785
min,3.233000e+03,0.000000,1385.000000,1381.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,2.490000,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,1.716403e+06,0.000000,1992.000000,1981.000000,0.000000,127.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,3.832500,6.570000,0.000000e+00,0.000000e+00,0.000000,0.000000
50%,3.680796e+06,0.000000,2000.000000,1994.000000,0.000000,214.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,9.990000,15.290000,0.000000e+00,0.000000e+00,0.000000,0.000000
75%,6.143775e+06,0.000000,2006.000000,2002.750000,0.000000,327.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,15.907500,21.530000,0.000000e+00,0.000000e+00,0.039525,0.017647
max,8.627540e+06,276883.000000,2013.000000,2010.000000,5.000000,2752.000000,100.000000,4.000000,13069.000000,1206.000000,61468.000000,2.691092e+06,78.360000,63.820000,4.136582e+06,1.081219e+06,2790.497076,631.158824


In [548]:
book_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7178 entries, 0 to 7177
Data columns (total 23 columns):
book_num         7178 non-null int64
title            7178 non-null object
author           7178 non-null object
followers        7178 non-null int64
pub_date         319 non-null float64
og_pub_date      166 non-null float64
avg_rating       7178 non-null float64
genre            7174 non-null object
binding          7178 non-null object
pages            307 non-null float64
language         7178 non-null object
perc_like        7178 non-null int64
trivia           7178 non-null int64
quote_likes      7178 non-null int64
rev_likes        7178 non-null int64
num_revs         7178 non-null int64
num_ratings      7178 non-null int64
kindle_price     58 non-null float64
amzn_price       13 non-null float64
total_added      7178 non-null int64
total_to_read    7178 non-null int64
avg_added        7178 non-null float64
avg_to_read      7178 non-null float64
dtypes: float64(8), int64(10

In [549]:
# sns.pairplot(book_df.drop(['book_num','trivia'], axis=1));

In [550]:
mask = ((book_df.title != 'Unknown') & (book_df.author != 'Unknown'))
book_df = book_df[mask]

In [551]:
book_df.binding = book_df.binding\
.replace(['Kindle Edition', 'e book', 'Nook'], 'ebook')\
.replace('Mass Market Paperback', 'Paperback')\
.replace(['Audiobook', 'Audio CD', 'Audio Cassette'], 'Audio')

binding_count = book_df.binding.value_counts()
other = list(binding_count[binding_count <= 7].index)
book_df.binding = book_df.binding.replace(other, 'Other')

book_df.binding.value_counts()

Paperback          156
Hardcover          126
Unknown Binding     67
ebook               16
Audio               11
Other                6
Name: binding, dtype: int64

In [552]:
genre_count = book_df.genre.value_counts()
other = list(genre_count[genre_count <= 2].index)

book_df.genre = book_df.genre.replace(other, 'Other')
book_df.genre.value_counts()

Unknown           235
Other              39
Nonfiction         12
History            12
Fiction            10
Fantasy             6
Historical          6
Religion            6
Art                 5
Childrens           5
Poetry              4
Science             4
Sequential Art      4
Philosophy          4
Classics            4
Mystery             4
Business            3
Reference           3
Christian           3
Plays               3
Biography           3
Psychology          3
Name: genre, dtype: int64

In [553]:
lang_count = book_df.language.value_counts()
other = list(lang_count[lang_count < 2].index)

book_df.language = book_df.language.replace(other, 'Other')
book_df.language.value_counts()

English       263
Unknown        95
Spanish        10
French          4
Other           4
Chinese         2
Indonesian      2
Dutch           2
Name: language, dtype: int64

In [554]:
book_df.followers = book_df.followers.apply(lambda x: np.log(x))
book_df = book_df.replace([np.inf, -np.inf], np.nan)
book_df.followers = book_df.followers.fillna(0)

In [555]:
book_df.quote_likes = book_df.quote_likes.apply(lambda x: np.log(x))
book_df = book_df.replace([np.inf, -np.inf], np.nan)
book_df.quote_likes = book_df.quote_likes.fillna(0)

In [556]:
book_df.rev_likes = book_df.rev_likes.apply(lambda x: np.log(x))
book_df = book_df.replace([np.inf, -np.inf], np.nan)
book_df.rev_likes = book_df.rev_likes.fillna(0)

In [557]:
book_df.num_revs = book_df.num_revs.apply(lambda x: np.log(x))
book_df = book_df.replace([np.inf, -np.inf], np.nan)
book_df.num_revs = book_df.num_revs.fillna(0)

In [558]:
book_df['likes_per_rev'] = book_df.rev_likes / book_df.num_revs
book_df = book_df.replace([np.inf, -np.inf], np.nan)
book_df.likes_per_rev = book_df.likes_per_rev.fillna(0)

In [576]:
feature_select = ['followers', 'avg_added', 'total_added', 'quote_likes',
                  'rev_likes', 'total_to_read', 'num_ratings', 'num_revs',
                  'likes_per_rev']

remain = book_df.drop(feature_select, axis=1).columns
target = book_df.avg_to_read 
features = book_df[feature_select]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [577]:
lm = LinearRegression()
lm.fit(X_train, y_train)
lm.score(X_test, y_test) # score on test data

0.9656542670084843

In [578]:
lm.score(X_train, y_train) # score on training data

0.9997055017447369

In [579]:
list(zip(X_train.columns, lm.coef_))

[('followers', -0.012989033733496868),
 ('avg_added', 0.23973202804653887),
 ('total_added', 3.685778902679444e-05),
 ('quote_likes', 0.03790262559465882),
 ('rev_likes', -0.048795200540695464),
 ('total_to_read', 0.00015203825328827502),
 ('num_ratings', -8.943566053757474e-05),
 ('num_revs', -0.014242264785987146),
 ('likes_per_rev', 0.3684177679842617)]

In [580]:
book_df['res'] = abs(lm.predict(book_df[feature_select]) - target)
book_df.head()

,book_num,title,author,followers,pub_date,og_pub_date,avg_rating,genre,binding,pages,...,num_revs,num_ratings,kindle_price,amzn_price,total_added,total_to_read,avg_added,avg_to_read,likes_per_rev,res
0,4158881,Tribe of Women: A Photojournalist Chronicles t...,Connie Bickman,0.0,2001.0,NaN,4.56,Unknown,Hardcover,160.0,...,0.0,9,NaN,16.67,19,8,0.005882,0.005882,0.0,0.009645
1,3164992,From Bento To Mixed Plate: Americans Of Japane...,Arnold T. Hiura,0.0,NaN,NaN,4.00,Unknown,Unknown Binding,215.0,...,0.0,1,NaN,NaN,4,0,0.000000,0.000000,0.0,0.013063
2,7758777,Mental Incarceration,Jullian Smallwood,0.0,2009.0,NaN,5.00,Unknown,Paperback,300.0,...,0.0,1,NaN,19.99,0,0,0.000000,0.000000,0.0,0.012916
3,3588329,A New Treatise on Accompaniment: With the Harp...,Michel De Saint-Lambert,0.0,1991.0,NaN,0.00,Unknown,Paperback,180.0,...,0.0,0,NaN,NaN,0,0,0.000000,0.000000,0.0,0.013005
4,3743805,Modern art in prints,Riva Castleman,0.0,1973.0,1973.0,0.00,Unknown,Unknown Binding,75.0,...,0.0,0,NaN,NaN,0,0,0.000000,0.000000,0.0,0.013005


In [581]:
def split_and_validate(X, y):
    '''
    For a set of features and target X, y, perform a 80/20 train/val split, 
    fit and validate a linear regression model, and report results
    '''
    
    # perform train/val split
    X_train, X_val, y_train, y_val = \
        train_test_split(X, y, test_size=0.2, random_state=420)
    
    # fit linear regression to training data
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    
    # score fit model on validation data
    val_score = lr_model.score(X_val, y_val)
    
    # report results
    print('\nValidation R^2 score was:', val_score)
    print('Feature coefficient results: \n')
    for feature, coef in zip(X.columns, lr_model.coef_):
        print(feature, ':', f'{coef:.2f}') 

In [582]:
feature_select.extend(['language'])
split_and_validate(pd.get_dummies(book_df[feature_select], drop_first=True), target)


Validation R^2 score was: 0.9972385512051657
Feature coefficient results: 

followers : -0.01
avg_added : 0.36
total_added : 0.00
quote_likes : 0.05
rev_likes : -0.25
total_to_read : 0.00
num_ratings : -0.00
num_revs : -0.01
likes_per_rev : 0.93
language_Dutch : 0.03
language_English : -0.00
language_French : 0.01
language_Indonesian : -0.06
language_Other : 0.01
language_Spanish : -0.06
language_Unknown : 0.02


In [583]:
kf = KFold(n_splits=5, shuffle=True, random_state = 420)
scores = cross_val_score(lm, pd.get_dummies(book_df[feature_select], drop_first=True), target, cv=kf, scoring='r2')
np.mean(scores)

0.9195849425725775